In [24]:
!pip install pandas python-docx pdfplumber openpyxl xlrd pymupdf

   ---------------------------------------- 0.0/18.4 MB ? eta -:--:--
   ---------- ----------------------------- 5.0/18.4 MB 26.8 MB/s eta 0:00:01
   ----------------------- ---------------- 10.7/18.4 MB 26.7 MB/s eta 0:00:01
   ----------------------------------- ---- 16.5/18.4 MB 26.9 MB/s eta 0:00:01
   ---------------------------------------- 18.4/18.4 MB 24.2 MB/s eta 0:00:00


In [2]:
# OCR 파트
import json
import time
import requests
from pathlib import Path
import re
import fitz  # PyMuPDF

import os
from dotenv import load_dotenv

load_dotenv()

INVOKE_URL = os.getenv("CLOVA_INVOKE_URL")
SECRET_KEY = os.getenv("CLOVA_OCR_SECRET")

# 파일(이미지/페이지)을 bytes 형태로 Clova OCR API에 보내서 OCR 결과 JSON을 받아옴
def _call_clova_from_bytes(
    filename: str,
    file_bytes: bytes,
    fmt: str,
    lang: str = "ko",
    enable_table: bool = True,
    timeout: int = 180,
) -> dict:
    headers = {"X-OCR-SECRET": SECRET_KEY}
    message = {
        "version": "V1",
        "requestId": f"clova-{Path(filename).stem}-{int(time.time()*1000)}",
        "timestamp": int(time.time() * 1000),
        "lang": lang,
        "images": [{"format": fmt, "name": Path(filename).stem}],
        "enableTableDetection": bool(enable_table),
    }
    files = {
        "file": (filename, file_bytes, "application/octet-stream"),
        "message": (None, json.dumps(message), "application/json"),
    }
    r = requests.post(INVOKE_URL, headers=headers, files=files, timeout=timeout)
    r.raise_for_status()
    return r.json()

# PDF를 페이지별로 렌더링해서 PNG 이미지 바이트로 변환
def _pdf_to_png_bytes_list(pdf_path: str, dpi: int = 200) -> list[tuple[str, bytes]]:
    p = Path(pdf_path)
    doc = fitz.open(str(p))
    zoom = dpi / 72
    mat = fitz.Matrix(zoom, zoom)

    out: list[tuple[str, bytes]] = []
    for i in range(len(doc)):
        pix = doc[i].get_pixmap(matrix=mat, alpha=False)
        out.append((f"{p.stem}_p{i+1}.png", pix.tobytes("png")))
    doc.close()
    return out

# Clova가 준 테이블 셀 1개에서 텍스트를 뽑는데, 셀 안에 여러 줄이면 줄바꿈(\n)을 유지해서 합침
def _cell_text_keep_lines(cell: dict) -> str:
    lines: list[str] = []

    if isinstance(cell.get("cellTextLines"), list) and cell["cellTextLines"]:
        for ln in cell["cellTextLines"]:
            if isinstance(ln.get("cellWords"), list) and ln["cellWords"]:
                s = " ".join(
                    [w.get("inferText", "") for w in ln["cellWords"] if w.get("inferText")]
                )
            else:
                s = ln.get("inferText", "")
            s = re.sub(r"\s+", " ", str(s)).strip()
            if s:
                lines.append(s)

    elif isinstance(cell.get("cellWords"), list) and cell["cellWords"]:
        s = " ".join([w.get("inferText", "") for w in cell["cellWords"] if w.get("inferText")])
        s = re.sub(r"\s+", " ", str(s)).strip()
        if s:
            lines.append(s)

    else:
        s = (cell.get("inferText") or cell.get("text") or "")
        s = re.sub(r"\s+", " ", str(s)).strip()
        if s:
            lines.append(s)

    return "\n".join(lines).strip()

# Clova OCR 결과 중에서 페이지의 일반 텍스트 영역(fields)을 전부 이어 붙여 페이지 전체 텍스트로 만듬
def _page_text_from_fields(result_json: dict) -> str:
    img0 = (result_json.get("images") or [{}])[0]
    fields = img0.get("fields") or []
    return " ".join([f.get("inferText", "") for f in fields if f.get("inferText")]).strip()

# 테이블에서 rowSpan / colSpan(병합셀) 정보를 보고, 병합된 셀의 텍스트를 병합 영역 전체 좌표에 복제해서 셀 리스트 만듬
def _merge_cells_with_rowspan(cells: list[dict]) -> list[dict]:
    has_span = any(("rowSpan" in c) or ("colSpan" in c) for c in cells)
    if not has_span:
        return cells

    expanded_cells: list[dict] = []
    for cell in cells:
        row_idx = cell.get("rowIndex", 0)
        col_idx = cell.get("columnIndex", 0)
        row_span = cell.get("rowSpan", 1) or 1
        col_span = cell.get("colSpan", 1) or 1
        text = _cell_text_keep_lines(cell)

        for r in range(row_idx, row_idx + row_span):
            for c in range(col_idx, col_idx + col_span):
                expanded_cells.append(
                    {
                        "rowIndex": r,
                        "columnIndex": c,
                        "text": text,
                        "isMerged": (r != row_idx or c != col_idx),
                    }
                )

    return expanded_cells

# Clova OCR 결과에서 tables를 꺼내서 2차원 배열(grid) 형태로 정리
def _tables_to_schema(result_json: dict) -> dict:
    img0 = (result_json.get("images") or [{}])[0]
    tables = img0.get("tables") or []

    out: dict = {}
    for ti, t in enumerate(tables, start=1):
        cells = t.get("cells") or []
        if not cells:
            out[f"table{ti}"] = {"rows": [], "rowCount": 0, "colCount": 0}
            continue

        has_rc = any("rowIndex" in c for c in cells) and any("columnIndex" in c for c in cells)
        if not has_rc:
            texts = [_cell_text_keep_lines(c) for c in cells]
            texts = [x for x in texts if x]
            rows = [[x] for x in texts]
            out[f"table{ti}"] = {"rows": rows, "rowCount": len(rows), "colCount": 1 if rows else 0}
            continue

        expanded_cells = _merge_cells_with_rowspan(cells)

        max_r = max(c.get("rowIndex", 0) for c in expanded_cells)
        max_c = max(c.get("columnIndex", 0) for c in expanded_cells)
        grid = [[""] * (max_c + 1) for _ in range(max_r + 1)]

        for c in expanded_cells:
            r = c.get("rowIndex", 0)
            col = c.get("columnIndex", 0)
            text = c.get("text", "")
            if not grid[r][col]:
                grid[r][col] = text

        out[f"table{ti}"] = {
            "rows": grid,
            "rowCount": len(grid),
            "colCount": len(grid[0]) if grid else 0,
        }

    return out

# 전체 파이프라인 엔트리 함수
def clovaOCR(file_path: str, lang: str = "ko", dpi: int = 200, enable_table: bool = True) -> dict:
    p = Path(file_path)
    if not p.exists():
        raise FileNotFoundError(str(p.resolve()))

    pages_out: dict = {}

    if p.suffix.lower() == ".pdf":
        page_pngs = _pdf_to_png_bytes_list(str(p), dpi=dpi)

        for i, (fname, png_bytes) in enumerate(page_pngs, start=1):
            print(f"[{i}/{len(page_pngs)}] OCR...") # OCR 되는지 확인 용 디버깅
            rj = _call_clova_from_bytes(
                fname, png_bytes, fmt="png", lang=lang, enable_table=enable_table
            )
            tables = _tables_to_schema(rj)

            pages_out[f"page{i}"] = {
                "text": _page_text_from_fields(rj),
                "tableCount": len(tables),
                "tables": tables,
            }

        return {"pageCount": len(page_pngs), "pages": pages_out}

    with open(p, "rb") as f:
        b = f.read()

    fmt = p.suffix.lower().lstrip(".")
    if fmt == "jpeg":
        fmt = "jpg"

    rj = _call_clova_from_bytes(p.name, b, fmt=fmt, lang=lang, enable_table=enable_table)
    tables = _tables_to_schema(rj)

    pages_out["page1"] = {
        "text": _page_text_from_fields(rj),
        "tableCount": len(tables),
        "tables": tables,
    }
    return {"pageCount": 1, "pages": pages_out}


In [3]:
from pathlib import Path
import pandas as pd
from docx import Document
import pdfplumber

CSV_EXT = {"csv"}
EXCEL_EXT = {"xls", "xlsx"}
IMAGE_EXT = {"jpg", "png"}

# -----------------------------
# 공통 출력 빌더 (handle_file에서만 호출)
# -----------------------------
def build_output(
    slotName: str,
    kind: str,
    ext: str,
    period_start: str | None,
    period_end: str | None,
    payload: dict | None = None,
):
    out = {
        "slotName": slotName,
        "kind": kind,
        "ext": ext,
        "period_start": period_start,
        "period_end": period_end,
    }
    if payload:
        out.update(payload)  # mode/content/dataframe 등 합치기
    return out


# -----------------------------
# 핸들러들은 "payload만" 반환
# -----------------------------
def handle_csv(path: str):
    for enc in ["euc-kr", "cp949", "utf-8"]:
        try:
            df = pd.read_csv(path, encoding=enc)
            return {"dataframe": df}
        except UnicodeDecodeError:
            continue
    raise ValueError("인코딩 실패: cp949, euc-kr, utf-8 encoding을 지원합니다")


def handle_excel(path: str):
    df = pd.read_excel(path)
    return {"dataframe": df}


def handle_image(path: str):
    ocr_output = clovaOCR(path)
    return {"mode": "ocr", "content": ocr_output}


def clean_table(tbl):
    return [
        [c for c in ("" if c is None else str(c).strip() for c in row) if c]
        for row in tbl
        if any("" if c is None else str(c).strip() for c in row)
    ]


def handle_pdf(path: str):
    PDF_PAGE_MIN_CHARS = 30
    PDF_PASS_RATIO = 0.7

    with pdfplumber.open(path) as pdf:
        total_pages = len(pdf.pages)
        valid_page_count = 0
        pages_data = {}

        for i, page in enumerate(pdf.pages):
            t = page.extract_text() or ""
            clean_t = " ".join(t.split()).strip()

            if len(clean_t) >= PDF_PAGE_MIN_CHARS:
                valid_page_count += 1

            raw = page.extract_tables() or []
            tables = {}
            for ti, tbl in enumerate(raw, start=1):
                rows = clean_table(tbl)
                tables[f"table{ti}"] = {
                    "rows": rows,
                    "rowCount": len(rows),
                    "colCount": max((len(r) for r in rows), default=0),
                }

            pages_data[f"page{i+1}"] = {
                "text": clean_t,
                "tableCount": len(raw),
                "tables": tables,
            }

        pass_ratio = valid_page_count / max(total_pages, 1)

    # text 모드
    if pass_ratio >= PDF_PASS_RATIO:
        return {
            "mode": "text",
            "content": {"pageCount": total_pages, "pages": pages_data},
        }

    # ocr 모드
    ocr_output = clovaOCR(path)
    return {"mode": "ocr", "content": ocr_output}


def handle_docx(path: str):
    doc = Document(path)
    paragraphs = [p.text.strip() for p in doc.paragraphs if p.text.strip()]

    tables = {}
    for i, table in enumerate(doc.tables, start=1):
        rows = [[cell.text for cell in row.cells] for row in table.rows]
        tables[f"table{i}"] = {
            "rows": rows,
            "rowCount": len(rows),
            "colCount": max((len(r) for r in rows), default=0),
        }

    pages = {
        "page1": {
            "text": "\n".join(paragraphs),
            "tableCount": len(doc.tables),
            "tables": tables,
        }
    }
    return {"content": {"pageCount": 1, "pages": pages}}


# -----------------------------
# 파일 형식별 조건부 함수 구동 (여기서 build_output 호출)
# -----------------------------
def handle_file(path: str, slotName: str, period_start: str | None, period_end: str | None):
    ext = Path(path).suffix.lower().lstrip(".")

    if ext in CSV_EXT:
        payload = handle_csv(path)
        return build_output(slotName, "CSV", "csv", period_start, period_end, payload)

    if ext in EXCEL_EXT:
        payload = handle_excel(path)
        return build_output(slotName, "EXCEL", ext, period_start, period_end, payload)

    if ext in IMAGE_EXT:
        payload = handle_image(path)
        return build_output(slotName, "IMAGE", ext, period_start, period_end, payload)

    if ext == "pdf":
        payload = handle_pdf(path)
        return build_output(slotName, "PDF", ext, period_start, period_end, payload)

    if ext == "docx":
        payload = handle_docx(path)
        return build_output(slotName, "DOCX", ext, period_start, period_end, payload)

    raise ValueError(f"지원하지 않는 확장자: {ext}")


In [8]:
# 1. DOCX 테스트
path = "test_file/docx_test1.docx" 
test_result = handle_file(path, "slot_name", "2025-11-01T00:00:00", "2025-11-15T23:59:59")
display(test_result)

{'slotName': 'slot_name',
 'kind': 'DOCX',
 'ext': 'docx',
 'period_start': '2025-11-01T00:00:00',
 'period_end': '2025-11-15T23:59:59',
 'content': {'pageCount': 1,
  'pages': {'page1': {'text': '공급망 ESG 진단 산출물\n(진단서 + 위험군 보고서)\n※ 이 문서는 실제 제출용이 아닌 시연용(임시) 예시입니다. 실제 서비스에서는 업종/규모/증빙 업로드 결과에 따라 항목·결과가 자동 생성됩니다.\n목차\n위험군 보고서\n진단서\n1. 위험군 보고서\n1.1 등급\n종합 등급 : B (중 위험군)\n판정 근거(요약): 필수 항목 중 미충족 3건 및 미확인/누락 다수 존재\n1.2 결과(10줄 이상, 세부)\n정성 체크리스트 25개 중 충족 6개, 부분충족 11개, 미충족 3개, 미확인 5개로 집계됨(샘플).\n인권·노동 영역에서 차별·괴롭힘 금지 및 신고채널 운영이 미충족으로 확인되어 인권 리스크가 상향됨.\n강제노동 금지 절차는 일부 존재하나(부분충족), 해외/이주노동자 관련 증빙이 없어 검증 강도가 낮음.\n근로시간 관리는 일부 기록으로 확인되나, 초과근로 통제(사전 승인/상한 관리) 체계가 불충분함.\n안전보건 영역에서 산업재해·아차사고 보고/재발방지 프로세스가 미충족으로 확인되어 중대사고 예방 체계가 취약함.\n위험성평가는 일부 수행 기록이 있으나, 개선조치 이행 증빙(CAPA 완료 증빙)이 부족하여 효과 검증이 어려움.\n외주/협력 인력의 안전교육 이수율 데이터가 누락되어, 현장 작업자 전체에 대한 안전 통제가 확인되지 않음.\n환경 영역은 폐기물 인계서로 기본 준수는 확인되지만, 인허가·법정 측정 성적서가 미제출되어 규제 리스크를 정량 평가하기 어려움.\n온실가스(Scope 1+2) 및 용수 사용량 등 핵심 환경 데이터가 누락되어, 고객사 요구 또는 공시 대응에 제약이 발생할 수 있음.\n윤리·컴

In [9]:
# 2. PDF 테스트 (OCR 유도용)
path = "test_file/pdf_ocr_test1.pdf" 
test_result = handle_file(path, "slot_name", "2025-11-01T00:00:00", "2025-11-15T23:59:59")
display(test_result)

[1/1] OCR...


{'slotName': 'slot_name',
 'kind': 'PDF',
 'ext': 'pdf',
 'period_start': '2025-11-01T00:00:00',
 'period_end': '2025-11-15T23:59:59',
 'mode': 'ocr',
 'content': {'pageCount': 1,
  'pages': {'page1': {'text': "HSB Registration Services Certificate of Registration This is to certify that: Sungkwang Bend Co., Ltd. Head Office: 26, Noksansandan 262-ro. Gangseo-Gu, Busan. Korea 2nd Factory: 35, Noksansandan 262-ro. Gangseo-gu, Busan, Korea 3rd Factory: 9, Noksansanecpbuk-ro 221beon-gil, Gangseo-gu, Busan, Korea Has established and applied an Occupational Health and Safety Management System for: Head Office: Manufacturing of Pipe Fittings and Flanges 2nd Factory: Manufacturing of Pipe Fittings 3rd Factory: Manufacturing of Pipe Fittings (Roll Bending & Welding) Proof has been furnished that the requirements according to ISO 45001:2018 are fulfilled. 1 ancingnalslu Certificate Number: 0 127 IAF Code(s): 17 Signed on behalf of HSB Registration Services Initial Audit Date: June 11, 2010 Effec

In [10]:
# 3. PDF 테스트 (일반 파싱용 1)
path = "test_file/pdf_plumber_test1.pdf" 
test_result = handle_file(path, "slot_name", "2025-11-01T00:00:00", "2025-11-15T23:59:59")
display(test_result)

{'slotName': 'slot_name',
 'kind': 'PDF',
 'ext': 'pdf',
 'period_start': '2025-11-01T00:00:00',
 'period_end': '2025-11-15T23:59:59',
 'mode': 'text',
 'content': {'pageCount': 1,
  'pages': {'page1': {'text': 'VI. 이사회 등 회사의 기관에 관한 사항 1. 이사회에 관한 사항 가. 이사회 구성 개요 사업보고서 제출일 현재 당사의 이사회는 3명의 상근이사, 1명의 사외이사 총 4명의 이사로 구성되어 있습니다. ⇒ 각 이사의 주요 이력 및 업무분장은 VIII. 임원 및 직원 등에 관한 사항을 참조하시기 바랍 니다. (1) 사외이사 및 그 변동현황 (단위 : 명) 사외이사 변동현황 이사의 수 사외이사 수 선임 해임 중도퇴임 4 1 - - - ※ 이사의 수는 사외이사를 포함한 총 이사의 수입니다. 나. 주요 의결사항 사내 사내 사외 이사 이사 이사 가 안 재 안 갑 김 재 회 개최일 결 일 원 호 의 안 내 용 차 자 여 (출 (출 (출 부 석률: 석률: 석률: 100 100 100 %) %) %) 2024.0 신규채무약정 1건에 관한 건 가 1 찬성 찬성 찬성 1.19 결 2024.0 내부회계관리제도 운영실태 보고의 건 가 2 찬성 찬성 찬성 2.05 결 2024.0 내부회계관리제도 운영실태 평가 감사보고의 건 가 3 찬성 찬성 찬성 2.08 결 제44기(2023년도) 현금배당 결정의 건 - 배당금액 : 4,189,282,050원 2024.0 가 4 찬성 찬성 찬성 2.26 - 배당주식수 : 28,600,000주중 자기주식 보유분 결 671,453주를 제외한 27,928,547주 - 배 당 금 : 1주당 150원의 현금배당 예정',
    'tableCount': 2,
    'tables': {'table1': {'rows': [['이사의 수', '사외이사 수', '사외이사 변동현황'],
    

In [11]:
# 4. PDF 테스트 (일반 파싱용 2)
path = "test_file/pdf_plumber_test2.pdf" 
test_result = handle_file(path, "slot_name", "2025-11-01T00:00:00", "2025-11-15T23:59:59")
display(test_result)

{'slotName': 'slot_name',
 'kind': 'PDF',
 'ext': 'pdf',
 'period_start': '2025-11-01T00:00:00',
 'period_end': '2025-11-15T23:59:59',
 'mode': 'text',
 'content': {'pageCount': 1,
  'pages': {'page1': {'text': '11. 정보보안 영역에서 보안사고 대응 및 백업 정책이 미충족이며, 백업 주기 데이터도 누락되어 운영중단·데이터 손실 리스크가 큼. 12. 정량 항목 25개 중 누락 10개, 불확실 8개로 데이터 완성도가 낮아, 등급 산정 시 보수적으로(리스크 상향) 반영됨. 1.3 핵심 리스크 요인 Top 5 순위 리스크 요인 근거(연관 항목) 영향도 긴급도 1 보안사고 대응 및 정성-정보보안(미충족), 매우 높음 높음 백업 정책 부재 정량 QN-22(누락) 2 산업재해/아차사고 정성-안전보건(미충족), 높음 높음 재발방지 체계 미흡 정량 QN-03(누락) 3 익명 신고채널·보호 정성-인권(미충족), 정량 높음 중간 절차 미흡 QN-17(누락) 4 온실가스 배출량 정량 QN-07(누락), 중간 중간 산정 데이터 누락 에너지 모니터링(부분충족) 5 2차 협력사 ESG 정성- 중간 중간 평가/개선 추적 미흡 공급망(부분충족/미확인), 정량 QN-19~20(누락) 1.4 개선 권고(피드백) 권고ID 권고 내용 관련 리스크 우선순위 권장 담당(예시) 기한 R-01 익명 신고채널(웹/전화) 구축 및 인권/윤리 최상 30일 현대협력 신고자 보호 절차(비밀보장·불이익 금지·처리기한) 문서화 R-02 산업재해·아차사고 보고-조사- 안전보건 최상 45일 현대협력 재발방지 표준 프로세스 수립(원인분석, CAPA, 재발검증 포함) R-03 백업 정책 정보보안 최상 45일 현대협력 수립(대상/주기/보관/복구 테스트) 및 랜섬웨어 대응 4',
    'tableCount': 2,
    'tables': {'table1': 

In [12]:
# 5. CSV 테스트
path = "test_file/csv_test1.csv" 
test_result = handle_file(path, "slot_name", "2025-11-01T00:00:00", "2025-11-15T23:59:59")
display(test_result)

{'slotName': 'slot_name',
 'kind': 'CSV',
 'ext': 'csv',
 'period_start': '2025-11-01T00:00:00',
 'period_end': '2025-11-15T23:59:59',
 'dataframe':                   癤풼ate  Usage_kWh  Lagging_Current_Reactive.Power_kVarh  \
 0      01/01/2018 00:15       3.17                                  2.95   
 1      01/01/2018 00:30       4.00                                  4.46   
 2      01/01/2018 00:45       3.24                                  3.28   
 3      01/01/2018 01:00       3.31                                  3.56   
 4      01/01/2018 01:15       3.82                                  4.50   
 ...                 ...        ...                                   ...   
 35035  31/12/2018 23:00       3.85                                  4.86   
 35036  31/12/2018 23:15       3.74                                  3.74   
 35037  31/12/2018 23:30       3.78                                  3.17   
 35038  31/12/2018 23:45       3.78                                  3.06   
 3503

In [13]:
# 6. XLSX 테스트
path = "test_file/xlsx_test1.xlsx" 
test_result = handle_file(path, "slot_name", "2025-11-01T00:00:00", "2025-11-15T23:59:59")
display(test_result)

{'slotName': 'slot_name',
 'kind': 'EXCEL',
 'ext': 'xlsx',
 'period_start': '2025-11-01T00:00:00',
 'period_end': '2025-11-15T23:59:59',
 'dataframe':                    date  Usage_kWh  Lagging_Current_Reactive.Power_kVarh  \
 0      01/01/2018 00:15       3.17                                  2.95   
 1      01/01/2018 00:30       4.00                                  4.46   
 2      01/01/2018 00:45       3.24                                  3.28   
 3      01/01/2018 01:00       3.31                                  3.56   
 4      01/01/2018 01:15       3.82                                  4.50   
 ...                 ...        ...                                   ...   
 35035  31/12/2018 23:00       3.85                                  4.86   
 35036  31/12/2018 23:15       3.74                                  3.74   
 35037  31/12/2018 23:30       3.78                                  3.17   
 35038  31/12/2018 23:45       3.78                                  3.06   
 3

In [14]:
# 7. JPG 테스트
path = "test_file/jpg_test1.jpg" 
test_result = handle_file(path, "slot_name", "2025-11-01T00:00:00", "2025-11-15T23:59:59")
display(test_result)

{'slotName': 'slot_name',
 'kind': 'IMAGE',
 'ext': 'jpg',
 'period_start': '2025-11-01T00:00:00',
 'period_end': '2025-11-15T23:59:59',
 'mode': 'ocr',
 'content': {'pageCount': 1,
  'pages': {'page1': {'text': "HSB Registration Services Certificate of Registration This is to certify that: Sungkwang Bend Co., Ltd. Head Office: 26, Noksansandan 262-ro, Gangsec-Gu, Busan, Korea 2nd Factory: 35, Noksansandan 262-ro, Gangseo-gu, Busan, Korea 3rd Factory: 9, Noksansanecpbuk-ro 221beon-gil, Gangseo-gu, Busan, Korea Has established and applied an Occupational Health and Safety Management System for: Head Office: Manufacturing of Pipe Fittings and Flanges 2nd Factory: Manufacturing of Pipe Fittings 3rd Factory: Manufacturing of Pipe Fittings (Roll Bending & Welding) Proof has been furnished that the requirements according to ISO 45001:2018 are fulfilled. GaneimonSalslu Certificate Number: O 127 IAF Code(s): 17 Signed on behalf of HSB Registration Services Initial Audit Date: June 11, 2010 Eff